<a href="https://colab.research.google.com/github/aidyai/lora-vit40k/blob/main/datasets/utils/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [6]:
import os
import json
import random
from PIL import Image
import pandas as pd
import wandb

import numpy as np
from pathlib import Path
from tqdm import tqdm

from shutil import copyfile

In [3]:
# Login with your autorization key
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!wget "https://huggingface.co/datasets/aidystark/FOOT40K/resolve/main/FOOT40k.tar.gz?download=true"
!wget "https://huggingface.co/datasets/aidystark/FOOT40K/resolve/main/foot40k.json?download=true"

--2024-02-27 23:01:50--  https://huggingface.co/datasets/aidystark/FOOT40K/resolve/main/FOOT40k.tar.gz?download=true
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.12, 65.8.178.93, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/21/c6/21c6a561f6c396c222a1d2b05fd063e87a278bb0668e9caa4513f74d34653686/db52b59949c5c5f8a4b2c2365662ee8125d96b5c256fd5c2c30fb9259321cfe7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27FOOT40k.tar.gz%3B+filename%3D%22FOOT40k.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1709334110&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTMzNDExMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzIxL2M2LzIxYzZhNTYxZjZjMzk2YzIyMmExZDJiMDVmZDA2M2U4N2EyNzhiYjA2NjhlOWNhYTQ1MTNmNzRkMzQ2NTM2ODYvZGI1MmI1OTk0OWM1YzVmOGE0YjJj

In [9]:
## Get dataset from URL
#Images & #Labels
image_files= "./FOOT40k.tar.gz"
label_files = "./foot40k.json"

In [ ]:

#extract tar files
#!tar -xzvf "./FOOT40k.tar.gz"
print(len(os.listdir("./FOOT40k")))

In [10]:
# Load JSON data into a DataFrame
df = pd.read_json(label_files)

# Save DataFrame to CSV
df.to_csv('train.csv', index=False)

Conversion completed. CSV file saved as output.csv


In [ ]:
IMG_RES = 256
WANDB_PROJECT = 'foot-net'
RAW_DATA_AT = f'raw_data_{IMG_RES}'

In [ ]:
# Read csv file
df = pd.read_csv('train.csv')

# Integer encode multi-label labels.
labels = list(df['labels'].value_counts().keys())
labels_dict = dict(zip(labels, range(12)))
id_to_labels = {val: key for key, val in labels_dict.items()}

df = df.replace({"labels": labels_dict})
df.head()

In [ ]:
# Initialize a new W&B run
run = wandb.init(project=WANDB_PROJECT, job_type="upload")

# create an artifact for all the raw data
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")
# Upload raw csv file as artifact
raw_data_at.add_file('train.csv')

# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label"])


In [ ]:
for i in tqdm(range(len(df))):
    image_name = df.loc[i]['image']
    label = df.loc[i]['labels']
    full_path = TRAIN_PATH+image_name

    # Append each example as a new row in Table.
    preview_dt.add_data(image_name, wandb.Image(full_path), id_to_labels[label])


In [ ]:
# save artifact to W&B
raw_data_at.add(preview_dt, "data_split")
run.log_artifact(raw_data_at)
run.finish()